In [ ]:
class Colorizer:
    def __init__(self, vmin=0, vmax=1, cmap_name = 'viridis'):
        cmap = plt.get_cmap(cmap_name)
        import matplotlib.colors as mcolors
        norm = mcolors.Normalize(vmin=vmin, vmax=vmax)
        self.scalar_map = plt.cm.ScalarMappable(norm=norm, cmap=cmap)
        
    def __call__(self, value):
        return self.scalar_map.to_rgba(value)

def filt(x):
    return scipy.signal.savgol_filter(x, 64, 2) 

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(25, 5))
c = Colorizer(vmax=3200)
for fiber in range(0, 3200, 25):
    ax1.plot(MAT.I_det[fiber, :], color=c(fiber))

fig.colorbar(c.scalar_map, ax=ax1)    
c = Colorizer(vmax=121)
for e in range(0, 121, 8):
    ax2.plot(filt(MAT.I_det[:, e]), color=c(e))
fig.colorbar(c.scalar_map, ax=ax2);

In [ ]:
freq_f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 5))

ax1.plot(MAT.mat['Df120']['Ln'][0], MAT.mat['Df120']['Fn'][0] * 1e3, label="Fibers")
ax1.plot(MAT.mat['Df120']['Le'][0], MAT.mat['Df120']['Fe'][0] * 1e3, label="Electrodes")
ax1.set_xlabel("Position along BM [mm]")
abt.utils.frequency_ax(ax1)
ax1.legend()
ax1.grid()
ax1.set_title("Greenwood function")

electrodes = np.arange(1, 17)
gwf_min_thresh = MAT.greenwood_f[np.argmin(MAT.I_det[:,::8], axis=0)]

ax2.plot(electrodes, abt.utils.virtual_channel_frequencies(16), label="Electrodes")
ax2.plot(electrodes, gwf_min_thresh, label="GW freq min threshold Fiber")

ax2.legend()
ax2.grid()
ax2.set_title("AB Electrode spec")
ax2.set_xlabel("Electrode")
abt.utils.frequency_ax(ax2)


# # Remapping selected fibers
min_thresh_fiber_idx = np.argmin(MAT.I_det[:,:], axis=0)

freq_f = scipy.interpolate.interp1d(
    np.r_[0, np.argmin(MAT.I_det[:,::8], axis=0), 3200], 
    np.r_[0, abt.utils.virtual_channel_frequencies(16), 10000], 
    fill_value='extrapolate'
)

fidx = np.arange(0, 3200)
int_freq = freq_f(fidx)

# ax2.plot(fidx, int_freq)

gwf = MAT.greenwood_f.copy()
gwf_scaled = gwf * .6

selected_fibers_gw = np.array([abt.utils.find_nearest_idx(gwf, val) for val in MEL_SCALE])
selected_fibers_gw_scaled = np.array([abt.utils.find_nearest_idx(gwf_scaled, val) for val in MEL_SCALE])
selected_fibers_est = np.array([abt.utils.find_nearest_idx(int_freq, val) for val in MEL_SCALE])

p = ax3.plot(fidx, gwf, label="GW f")
ax3.scatter(selected_fibers_gw, gwf[selected_fibers_gw], color=p[0].get_color(), label='selected fibers')
p = ax3.plot(fidx, freq_f(fidx), label="Estimated freq")
ax3.scatter(selected_fibers_est, int_freq[selected_fibers_est], color=p[0].get_color(), label='selected fibers')

p = ax3.plot(fidx, gwf_scaled, label="Scaled GW f")
ax3.scatter(selected_fibers_gw_scaled, gwf_scaled[selected_fibers_gw_scaled], color=p[0].get_color(), label='selected fibers')


ax3.legend()
ax3.set_xlabel("fiber id")
abt.utils.frequency_ax(ax3)
ax3.grid()
ax3.set_title("GW f vs Est. Freq. based on min thres");

# # Using estimated fibers
selected_fibers = selected_fibers_est
selected_fibers_freq = int_freq[selected_fibers]

# # Using scaled Gw function
# selected_fibers = selected_fibers_gw_scaled
# selected_fibers_freq =  gwf_scaled[selected_fibers]

# # Using original Gw function
# # selected_fibers = selected_fibers_gw
# # selected_fibers_freq =  gwf[selected_fibers]

assert np.unique(selected_fibers).size == selected_fibers.size
